# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [55]:
items = pd.read_parquet("./goodsread/items.par")
events = pd.read_parquet("./goodsread/events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = list(set(users_train) & set(users_test))

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [5]:
cold_users = set(users_test) - set(common_users)

print(len(cold_users)) 

2365


In [12]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(by="popularity_weighted",ascending=False).reset_index()

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.loc[item_popularity["avg_rating"] >= 4][0:100]
top_k_pop_items

,index,item_id,users,avg_rating,popularity_weighted
2,32387,18007564,20207,4.321275,87320.0
3,32623,18143977,19462,4.290669,83505.0
4,30695,16096824,16770,4.301014,72128.0
5,2,3,15139,4.706057,71245.0
7,3718,38447,14611,4.232770,61845.0
...,...,...,...,...,...
131,19596,2767052,4361,4.413437,19247.0
133,32835,18293427,4674,4.092640,19129.0
134,378,3636,4667,4.098564,19128.0
135,33611,18966819,4361,4.374914,19079.0


In [13]:
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], 
    on="item_id"
    ).sort_values(by="popularity_weighted",ascending=False).copy()

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,87320.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,83505.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,72128.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,71245.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,61845.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
5,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,60421.0,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,59963.0,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,57605.0,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,18692431,"Nicola Yoon, David Yoon","Everything, Everything",2015,14121,4.071454,57493.0,"{'Young Adult': 5175, 'Romance': 3234, 'Contem..."
9,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,56717.0,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."


# === Знакомство: первые персональные рекомендации

In [16]:
cold_users_events_with_recs = events_test[events_test["user_id"].isin(cold_users)].merge(top_k_pop_items[["avg_rating","item_id"]], on="item_id", how="left")
cold_users_events_with_recs

,item_id,started_at,read_at,is_read,rating,is_reviewed,user_id,avg_rating
0,6900,2017-10-09,2017-10-13,True,4,False,1361610,NaN
1,12555,2017-09-21,2017-10-11,True,3,False,1361610,NaN
2,25899336,2017-09-12,2017-09-17,True,4,True,1361610,4.427261
3,21936809,2017-08-20,2017-08-24,True,4,True,1361610,NaN
4,6952,2017-09-18,2017-09-20,True,3,False,1361610,NaN
...,...,...,...,...,...,...,...,...
9667,252499,2017-09-30,2017-10-06,True,4,False,1178502,NaN
9668,51113,2017-09-25,2017-10-07,True,4,False,1253160,NaN
9669,16181775,2017-09-24,2017-09-25,True,3,False,1253160,NaN
9670,10210,2017-09-16,2017-09-24,True,5,False,1253160,NaN


In [20]:

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx][["user_id", "item_id", "rating", "avg_rating"]]
print("количество холдных пользователей прочитавших книги из top100 рекомендаций",len(cold_user_recs))
cold_user_recs.head()

количество холдных пользователей прочитавших книги из top100 рекомендаций 1912


,user_id,item_id,rating,avg_rating
2,1361610,25899336,4,4.427261
5,1338996,16096824,5,4.301014
8,1338996,18692431,5,4.071454
9,1338996,28763485,2,4.194663
15,1276025,38447,5,4.232770


In [19]:
print(len(cold_user_recs)/len(cold_users))

0.8084566596194503


In [21]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [22]:
cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Базовые подходы: коллаборативная фильтрация

sparsity= количество всех ячеек / количество пустых ячеек

 - Количество всех ячеек это количество пользователей * количество объъектов
 - количество всех оценок в таблице events - это непустое значение на пересечении пользователь-книга

In [30]:
sparsity = (1 - len(events)/(events['user_id'].count() * events['item_id'].count())) * 100
print("sparsity =",sparsity.round(5),"%")

sparsity = 99.99999 %


In [31]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set) 

In [34]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [35]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8289
MAE:  0.6474
0.8288711689059135 0.647437483750257


In [38]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

mae_rand = accuracy.mae(random_predictions)
print(mae_rand)

MAE:  1.0018
1.0017726877569562


# Факультативно

Удалите из events события для редких айтемов — таких, с которыми взаимодействовало менее N пользователей. Возьмите небольшое N, например 2–3 пользователя. Получите рекомендации, посчитайте метрики, оцените, как они изменились. Подумайте, с чем могут быть связаны такие изменения.

In [56]:
events["count_events_with_item"] = events.groupby(["item_id"])["user_id"].transform("count")
events = events[events["count_events_with_item"] >= 2].copy()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = list(set(users_train) & set(users_test))

print(len(users_train), len(users_test), len(common_users))

428219 123205 120840


In [58]:
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set) 

surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)

RMSE: 0.8287
MAE:  0.6473


In [39]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events.query("user_id == @user_id")['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"]) 

In [74]:
rec_list = get_recommendations_svd(1296647, items, events_train, svd_model) 
rec_list

,item_id,score
0,24812,5.000000
1,11221285,4.990293
2,54741,4.954057
3,23602722,4.939657
4,22037424,4.931492


In [73]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

user_id: 1077696
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
130,Ian Rankin,"Strip Jack (Inspector Rebus, #4)",2012-06-23,2012-07-03,3,"{'Mystery-Crime': 258, 'Mystery': 244, 'Fictio..."
131,Suzanne Collins,"Mockingjay (The Hunger Games, #3)",2012-06-23,2012-07-03,5,"{'Young Adult': 24130, 'Science Fiction-Dystop..."
132,Ian Rankin,"Hide and Seek (Inspector Rebus, #2)",2012-06-13,2012-06-23,4,"{'Mystery': 381, 'Mystery-Crime': 365, 'Fictio..."
133,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",2012-06-11,2012-06-23,5,"{'Young Adult': 30042, 'Fiction': 16754, 'Scie..."
134,Neil Gaiman,Neverwhere,2017-02-17,2017-02-24,5,"{'Fantasy': 16696, 'Fiction': 4885, 'Fantasy-U..."
135,"Neil Gaiman, Dave McKean",The Graveyard Book,2012-06-01,2012-06-11,3,"{'Fantasy': 12242, 'Young Adult': 4944, 'Ficti..."
136,Agatha Christie,"The Moving Finger (Miss Marple, #4)",2012-05-20,2012-05-24,3,"{'Mystery': 1762, 'Fiction': 410, 'Mystery-Cri..."
137,Jon Krakauer,Into the Wild,2012-02-04,2012-02-19,5,"{'Nonfiction': 7968, 'Biography': 2508, 'Adven..."
138,"Orson Scott Card, Stefan Rudnicki, Harlan Ellison","Ender's Game (Ender's Saga, #1)",2011-02-18,2012-02-04,4,"{'Science Fiction': 15315, 'Fiction': 7488, 'Y..."
139,Philippa Gregory,The White Queen (The Plantagenet and Tudor Nov...,2010-08-26,2011-02-15,3,"{'Historical-Historical Fiction': 4822, 'Histo..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,24812,4.972094,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
1,34733250,4.945748,"Laini Taylor, Jim Di Bartolo",Night of Cake & Puppets (Daughter of Smoke & B...,"{'Fantasy': 1278, 'Young Adult': 818, 'Romance..."
2,494320,4.917449,"Irene Gut Opdyke, Jennifer Armstrong",In My Hands: Memories of a Holocaust Rescuer,"{'Nonfiction': 303, 'World War II-Holocaust': ..."
3,16164271,4.896829,"Joe Hill, Gabriel Rodríguez","Locke & Key, Vol. 6: Alpha & Omega","{'Sequential Art-Graphic Novels': 1292, 'Horro..."
4,2363958,4.857136,João Guimarães Rosa,Grande Sertão: Veredas,"{'Fiction': 85, 'Classics': 69, 'Cultural-Braz..."


# === Базовые подходы: контентные рекомендации

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков